根据关键点特征进行比配

In [2]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np
import time
import random
#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
#cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
#cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
#cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);


'''
firstframe=None

count=-1
frame_list=[]
'''
count=0
frame_now=None
continue_pt=None
background=None

bf = cv2.BFMatcher(cv2.NORM_L2) 

while (camera.isOpened()):  
    
    ret,frame_now = camera.read()
    count += 1
    
    #提取第一帧做背景
    if count == 1:
        background=frame_now
    
    if not ret:  
        break
        
    #小小地作弊了一下，直接算物体已经出现的150帧-310帧
    if count < 150 :
        continue 
    if  count == 310 :
        break
    
    #为了更好的测试代码，把获取移动物体然后框框的代码注释掉了
    '''
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
        continue  
    
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  

    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect
    '''

    
    sift = cv2.xfeatures2d.SIFT_create() 
    kp_now, des_now = sift.detectAndCompute(frame_now, None)  
    
    #初始化，对于第一帧，把所有关键点和描述符加入到continue_pt和des中
    #continue_pt是一个二维数组，每一行都希望储存一组连续的关键点，有多少组储存多少行
    #描述符des是一个一维数组，因为我认为连续的关键点可以共用一个描述符
    if continue_pt == None :
        kp_now=np.array(kp_now)
        #将continue_pt升维后转为list方便操作
        continue_pt=kp_now.reshape(len(kp_now),1)
        continue_pt=continue_pt.tolist()
        des=des_now
        continue
        
   
    if count%3==0:
        
        #匹配continue_pt中的关键点和当前帧的关键点，通过描述符des和des_now来匹配
        matches = bf.knnMatch(des, des_now, k = 2) 
        goodMatch = []
        for m,n in matches:  
            if m.distance < 0.1*n.distance:  
                    goodMatch.append(m)
        print(len(goodMatch))
        p1 = [kpp.queryIdx for kpp in goodMatch]  
        p2 = [kpp.trainIdx for kpp in goodMatch]     
        
        count_index=0
        for i in range(len(des_now)):
            #对于匹配成功的关键点，将当前帧中匹配成功的关键点加入到continue_pt的对应行中
            if i in p2 :
                continue_pt[p1[count_index]].append(kp_now[i])
                count_index += 1
            else:
                #对于匹配失败的关键点，将当前帧中匹配失败的关键点另起一行，加入到continue_pt的末尾
                #同时更新des
                continue_pt.append([kp_now[i]])
                des=np.append(des,[des_now[i]],axis=0)
            

        '''
        cv2.imshow(u"Video", frame)  
        cv2.imshow(u"BW-Video", thresh)  
        cv2.imshow(u"DI-Video", frameDelta)

        key = cv2.waitKey(1)
        if key == ord("q"):  
            break
        '''

r=lambda:random.randint(0,255)
#依次将每一组的连续关键点画出来，即当该组元素数量大于1时，才画线，否则略过
for k in continue_pt:
    if len(k)< 3:
        continue
    else:
        print(len(k))
        (x1,y1)=(None,None)
        color=(r(),r(),r())
        post = np.int32([m.pt for m in k]) 
        for (x2,y2) in post :
            if (x1,y1) == (None,None) :
                (x1,y1)=(x2,y2)
                continue

            background=cv2.line(background, (x1,y1),(x2,y2), color,thickness=1)  
            (x1,y1)=(x2,y2)

        
cv2.imshow('1',background)
cv2.waitKey(0)          
camera.release()  
cv2.destroyAllWindows()  


1
3
5
4
3
3
3
3
3
3
3
1
0
2
1
0
1
1
0
4
1
3
2
1
2
1
0
0
2
2
4
2
10
3
2
4
3
7
1
3
3
3
3
3
2
1
2
7
3
5
0
1
2
3
5
6
5
3
3
3
3
3
3
4
3
3
3
3
3
3
3
4
4
4
3
3
3
3


以下的代码不用看
以下的代码不用看
以下的代码不用看
以下的代码不用看
以下的代码不用看
以下的代码不用看
以下的代码不用看

In [3]:
import cv2  
import numpy as np
import time
#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  
count=0

while (camera.isOpened()):  
    
    ret,frame_1 = camera.read()
    if not ret:  
        break
    count +=1
    if count == 150:
        break
        
while (camera.isOpened()):  
    
    ret,frame_2 = camera.read()
    if not ret:  
        break
    count +=1
    if count == 310:
        break

cv2.imshow('1',frame_1)
cv2.imshow('2',frame_2)
cv2.waitKey(0)          
camera.release()  
cv2.destroyAllWindows() 

In [7]:
import cv2  
import numpy as np
import time

a=[[1],[2],[3],[4]]
a[0].append(1)
a

[[1, 1], [2], [3], [4]]